In [11]:
!pip install transformers

     |████████████████████████████████| 1.4MB 5.6MB/s 
     |████████████████████████████████| 2.9MB 44.7MB/s 
     |████████████████████████████████| 890kB 54.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=6a4e67232e593e165b812cb9192cef3fdffdb593c8811b2debe7e45ab53581b8
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [12]:
import os
import random
import collections
from transformers import BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments
import torch
import pprint
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')
drive_path = '/content/drive/My Drive/text2image'

Mounted at /content/drive


In [3]:
categories = ['Салаты',
'Супы',
'Пироги',
'Торты',
'Печенье',
'Сэндвичи',
'Запеканка',
'Кексы',
'Блины',
'Хлеб',
'Котлеты',
'Каши',
'Напитки',
'Омлет',
'Оладьи',
'Мороженое',
'Фруктовые салаты',
'Пирожное',
'Пудинг',
'Лапша',
'Пюре',
'Шашлык',
'Булочки',
'Стейки',
'Сырники',
'Яичница',
'Конфеты',
'Плов',
'Соусы и маринады',
'Паштеты',
'Роллы',
'Тосты',
'Суфле',
'Соленья и консервация',
'Лазанья',
'Гамбургер',
'Жаркое']

In [4]:
def have_single_strategy(tags, categories):
    actual_categories = list(set(tags) & set(categories))
    if len(actual_categories) == 1:
      return actual_categories[0]
    return None
def first_from_tags_strategy(tags, categories):
    categories = set(categories)
    for tag in tags:
      if tag in categories:
        return tag
    return None
def first_from_categories_strategy(tags, categories):
    tags = set(tags)
    for category in categories:
      if category in tags:
        return category
    return None

In [5]:
print(have_single_strategy(['Лазанья'], ['Суфле','Лазанья']))
print(have_single_strategy(['Лазанья', 'Суфле'], ['Суфле','Лазанья']))
print(first_from_tags_strategy(['Лазанья','Суфле'], ['Суфле','Лазанья']))
print(first_from_categories_strategy(['Лазанья','Суфле'], ['Суфле','Лазанья']))

Лазанья
None
Лазанья
Суфле


In [7]:
if not os.path.exists('GDriveDL.py'):
    !wget https://raw.githubusercontent.com/matthuisman/gdrivedl/master/gdrivedl.py -O GDriveDL.py

os.makedirs('data', exist_ok=True)
if not os.path.exists('data/eda_ru.zip'): 
    !python GDriveDL.py https://drive.google.com/file/d/1CNIbj8_OuxQD74zt6JU4BUI8ngctutl9/view?usp=sharing data
    !unzip -q data/eda_ru.zip -d data

data/eda_ru.zip
[==================================================] 2257.57MB/2257.57MB


In [13]:
tokenizer = BertTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
rubert = BertForSequenceClassification.from_pretrained("DeepPavlov/rubert-base-cased", num_labels=len(categories))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
def get_recipe_text(row):
    title_text = row['title'] 
    ingredients_text = row['ingredients'].replace('|', ' ')
    steps_text = row['steps'].replace('|', ' ')
    return f"{title_text} {ingredients_text} {steps_text}"

In [16]:
data = pd.read_csv('./data/eda_ru_filtered_with_tags.csv', sep=',')
data["tags"]= data["tags"].str.split(pat="|")

data["full text"] = data.apply(lambda x: get_recipe_text(x), axis=1)

tag_selection_stratagy = have_single_strategy
#tag_selection_stratagy = first_from_tags_strategy
#tag_selection_stratagy = first_from_categories_strategy

data["main tag"] = data.apply(lambda x: tag_selection_stratagy(x['tags'], categories), axis=1)
data_without_tag = data[data['main tag'].isna()]
data_with_tag = data[data['main tag'].notna()]

train, test = train_test_split(data_with_tag, test_size=0.1, random_state=42, shuffle=True)

train_token = tokenizer(train['full text'].astype(str).values.tolist(), truncation=True, padding=True, max_length=512)
test_token = tokenizer(test['full text'].astype(str).values.tolist(), truncation=True, padding=True, max_length=512)

def get_labels(data, labels):
  return data["main tag"].apply(lambda x: labels.index(x)).values.tolist()

train_label = get_labels(train, categories)
test_label = get_labels(test, categories)

In [17]:
class FoodDataSet(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


train_dataset = FoodDataSet(train_token, train_label)
test_dataset = FoodDataSet(test_token, test_label)

In [18]:
training_args = TrainingArguments(
    output_dir='./results',          
    num_train_epochs=3,              
    per_device_train_batch_size=8,  
    per_device_eval_batch_size=32,  
    warmup_steps=500,               
    weight_decay=0.01,               
    logging_dir='./logs',            
    logging_steps=100,
)

In [19]:
trainer = Trainer(
    model=rubert,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [20]:
torch.cuda.empty_cache()
trainer.train()

Step,Training Loss
100,3.370037
200,2.737322
300,1.825287
400,1.111474
500,0.777024
600,0.545900
700,0.510793
800,0.433784
900,0.402452
1000,0.403397


TrainOutput(global_step=4887, training_loss=0.40147549500610774)

In [21]:
def predict(text):
    inputs = tokenizer([text], truncation=True, padding=True, max_length=512, return_tensors="pt").to(rubert.device)
    outputs = rubert(**inputs,labels=None)
    logits = outputs.logits.cpu().detach().numpy()
    categoryIndex = np.argmax(logits)
    return categories[categoryIndex]

In [22]:
predict('Пирожки')

'Пирожное'

In [23]:
for i, row in data_without_tag.sample(20).iterrows():
  print(row['title'], predict(row['full text']))

#for t in random.sample(data_without_tag['full text'].astype(str).values.tolist(), 20):
#  print(t, predict(t))

Кальмары с левантийскими специями Салаты
Ростбиф из вырезки Жаркое
Варенье из желтых слив Напитки
«Наполеон» Торты
Сосисочные спиральки Печенье
Парфе из рикотты с манго и лаймом Пирожное
Весенняя карбонара со спаржей и горошком Пироги
Ананасово-яблочная шарлотка Пироги
Воздушные творожные запеканки на манной крупе Запеканка
Мясо, запеченное с сыром и помидорами Жаркое
Цветная капуста, запеченная с сыром Запеканка
Капуста, обжаренная с грибами Салаты
Кофейные эклеры. Пирожное
Икра из кабачков Супы
Зерновые гренки с малиновым джемом Сэндвичи
Хлеб на закваске с фундуком и изюмом Хлеб
Форель, запеченная с мелиссой и лаймом с гарниром из шариков риса Супы
Груши в ароматном сиропе Соусы и маринады
Рулетики из индейки со шпинатом и сыром в беконе Сэндвичи
Курица с китайской рисовой лапшой и грибами шиитаке Лапша
